In [2]:
import OpenDartReader
import pandas as pd 
import os
import glob
import numpy as np

In [ ]:
# df = pd.read_csv('esg_financial_data_9_columns.csv', 
#                  dtype={'corp_code': str, 'stock_code': str})
# c_code = df['stock_code'].to_list()
# stock_code = [str(code) for code in c_code]

# def add_zero(c):
#     if len(c) < 6:
#         c = (6-len(c)) * '0' + c
#     return c

# st_code = list(map(add_zero, stock_code))
# df['stock_code'] = st_code

# c_code = df['corp_code'].to_list()
# corp_code = [str(code) for code in c_code]

# def add_zero(c):
#     if len(c) < 8:
#         c = (8-len(c)) * '0' + c
#     return c

# st_code = list(map(add_zero, corp_code))
# df['corp_code'] = st_code

In [ ]:
# api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)

df = pd.read_csv('ESG_data_intangible.csv', index_col=0, dtype={'corp_code': str,
                                                       'stock_code': str})
print(df.shape)
df.head()

In [ ]:
entire_codes = []

for f in glob.glob('./2020/fin_2020/*.csv'):
    code = f.split('\\')[1][:6]
    entire_codes.append(code)

code_missing = [code for code in list(df['stock_code'].unique()) if code not in entire_codes]

print('저장된 회사 수: ', len(entire_codes)) # 670개 데이터 존재
print('누락 회사 수: ', len(code_missing)) # 124개는 다시 호출할 필요 있음

### 무형자산 데이터 import

In [ ]:
# csv 파일 합치기

df_c = df.copy()

df_c['intangible'] = np.zeros(df_c.shape[0])

code_missing = [code for code in list(df['stock_code'].unique()) if code in entire_codes]

missing_codes_intang = []
for code in code_missing:
    try:
        docu = pd.read_csv(f'./2022/fin_2022/{code}_2022.csv', index_col=0)
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code]

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
    except:
        missing_codes_intang.append(code)

df_c['intangible'] = pd.to_numeric(df_c['intangible'])

In [ ]:
# 누락된 데이터는 OpenDartreader 다시 이용
# 사업 보고서 > 연결재무제표
re_codes = df_c[df_c['intangible'] == 0]['stock_code'].unique()

re = []
for code in re_codes:
    try: 
        lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
        report_num = lst[(lst['report_nm'].str.contains('사업보고서')) & (lst['report_nm'].str.contains('2022'))].iloc[0, 5]
        url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
        html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

        docu = pd.read_html(html)[1]

        docu = docu.rename(columns={'Unnamed: 0': 'tag'})
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code]

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
        print(f'{code}: good')
    except:
        re.append(code)
print(len(re))

In [ ]:
# 사업보고서 > 영업보고서

re_re = []
for code in re:
    try:
        lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
        report_num = lst[(lst['report_nm'].str.contains('사업보고서')) & (lst['report_nm'].str.contains('2022'))].iloc[0, 5]
        url = dart.sub_docs(report_num).iloc[1,1] #사업보고서 번호를 통해 url 요청
        # html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소

        docu = pd.read_html(html)[1]

        docu = docu.rename(columns={'Unnamed: 0': 'tag'})
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code]

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
        print(f'{code}: good')
    except:
        re_re.append(code)

In [ ]:
df_c[df_c['intangible'] == 0]

In [ ]:
df_c.to_csv('./ESG_data_intangible.csv', encoding='utf8', index = False)

### 현금배당성향 import

In [3]:
df = pd.read_csv('ESG_data_intangible.csv', index_col=0, dtype={'corp_code': str,
                                                       'stock_code': str})
print(df.shape)
df.head()

(2370, 9)


,corp_code,stock_code,연도,ESG,재무제표,부채총계,영업이익,법인세차감전 순이익,intangible
회사명,,,,,,,,,
기아,00106641,000270,2022,B+,"73,710,965,000,000","34,367,884,000,000","7,233,120,000,000","7,501,993,000,000",2.905980e+06
S-Oil,00138279,010950,2022,A,"19,613,135,000,000","11,129,666,000,000","3,405,166,000,000","2,898,542,000,000",1.064180e+05
풀무원,00155355,017810,2022,A,"2,049,217,755,236","1,502,570,984,984","26,331,500,743","-18,413,093,758",9.776635e+10
케이티,00190321,030200,2022,A,"40,980,681,000,000","22,565,958,000,000","1,690,088,000,000","1,894,067,000,000",3.129833e+06
SK,00181712,034730,2022,A+,"194,505,860,000,000","122,697,070,000,000","8,004,710,000,000","5,908,697,000,000",1.270902e+07


In [7]:
entire_codes = []

for f in glob.glob('./2022/cash_2022/*.csv'):
    code = f.split('\\')[1][:8]
    entire_codes.append(code)

code_missing = [code for code in list(df['corp_code'].unique()) if code not in entire_codes]

print('저장된 회사 수: ', len(entire_codes)) # 690개 데이터 존재
print('누락 회사 수: ', len(code_missing)) # 3 개는 다시 호출할 필요 있음

저장된 회사 수:  844
누락 회사 수:  3


In [30]:
df_s = df.copy()
df_s['ratio'] = np.nan
# df_s['ratio'] = ""
df_s.head()

,corp_code,stock_code,연도,ESG,재무제표,부채총계,영업이익,법인세차감전 순이익,intangible,ratio
회사명,,,,,,,,,,
기아,00106641,000270,2022,B+,"73,710,965,000,000","34,367,884,000,000","7,233,120,000,000","7,501,993,000,000",2.905980e+06,NaN
S-Oil,00138279,010950,2022,A,"19,613,135,000,000","11,129,666,000,000","3,405,166,000,000","2,898,542,000,000",1.064180e+05,NaN
풀무원,00155355,017810,2022,A,"2,049,217,755,236","1,502,570,984,984","26,331,500,743","-18,413,093,758",9.776635e+10,NaN
케이티,00190321,030200,2022,A,"40,980,681,000,000","22,565,958,000,000","1,690,088,000,000","1,894,067,000,000",3.129833e+06,NaN
SK,00181712,034730,2022,A+,"194,505,860,000,000","122,697,070,000,000","8,004,710,000,000","5,908,697,000,000",1.270902e+07,NaN


In [31]:
# 1단계 csv 파일 합치기

re_codes= []
for code in df_s['corp_code'].unique():
    try:
        docu = pd.read_csv(f'./2022/cash_2022/{code}_2022.csv', index_col=0)
        cash = docu[docu['se'].str.contains('현금배당성향')].iloc[0, 2:]
        
        data = df_s[df_s['corp_code'] == code]

        for i, (k,v) in enumerate(data.ratio.items()):
            df_s.loc[k, 'ratio'] = cash[i]
    except:
        re_codes.append(code)

In [33]:
print(sum(df_s['ratio'].isna()))
df_s

9


,corp_code,stock_code,연도,ESG,재무제표,부채총계,영업이익,법인세차감전 순이익,intangible,ratio
회사명,,,,,,,,,,
기아,00106641,000270,2022,B+,"73,710,965,000,000","34,367,884,000,000","7,233,120,000,000","7,501,993,000,000",2.905980e+06,27.00
S-Oil,00138279,010950,2022,A,"19,613,135,000,000","11,129,666,000,000","3,405,166,000,000","2,898,542,000,000",1.064180e+05,-
풀무원,00155355,017810,2022,A,"2,049,217,755,236","1,502,570,984,984","26,331,500,743","-18,413,093,758",9.776635e+10,43.90
케이티,00190321,030200,2022,A,"40,980,681,000,000","22,565,958,000,000","1,690,088,000,000","1,894,067,000,000",3.129833e+06,49.60
SK,00181712,034730,2022,A+,"194,505,860,000,000","122,697,070,000,000","8,004,710,000,000","5,908,697,000,000",1.270902e+07,195.50
...,...,...,...,...,...,...,...,...,...,...
아난티,00161781,025980,2020,D,"1,143,676,045,142","776,959,250,375","-31,723,745,964","-49,742,103,200",4.019670e+09,-
상상인,00306162,038540,2020,D,"3,605,932,131,664","2,914,390,922,520","94,052,923,307","87,696,218,805",5.470796e+10,-
씨젠,00788773,096530,2020,D,"1,094,574,623,631","448,168,695,845","676,189,736,451","669,138,824,047",6.713725e+09,7.80


In [ ]:
# 2단계: 함수 불러오기
# api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
# api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
dart = OpenDartReader(api_key)

for code in re_codes:
    print(code)
    docu = dart.report(code, '배당', 2022)
    
    cash = docu[docu['se'].str.contains('현금배당성향')].iloc[0, 2:]
        
    data = df_s[df_s['corp_code'] == code]

    for i, (k,v) in enumerate(data.ratio.items()):
        df_s.loc[k, 'ratio'] = cash[i]